First, we need to install the required packages, so let's install the required packages .

In [1]:
pip install sacremoses==0.0.53

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=421ee2944d554acd9bb6b7530316a100e696d79b96ab030c50ffe415308a959c
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.7 MB/s eta 0:00:00


In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [4]:
pip install torch torchvision torchaudio

Next, we import the necessary modules from the installed packages:

In [5]:
from datasets import load_dataset
from transformers import pipeline

we create a summarization pipeline using the pipeline function from the transformers package.

In [6]:
#  We specify 'cache_dir' to use predownloaded models
summarizer = pipeline(
    task = "summarization",
    model = "t5-small",
    min_length = 20,
    max_length = 50,
    truncation = True,
    model_kwargs={"cache_dir":'/Documents/Hugging_Face/'}
)

 task parameter is set to “summarization”, indicating that the pipeline’s task is text summarization .  
We set 'truncation=True' to truncate the input text if it exceeds the maximum length  
For this we have used the model "t5-small" model i.e. T5 (Text-to-Text Transfer Transformer) model and lets see the performance

In [7]:
# Ask the user for input
input_text = input("Enter the text you want to summarize: ")

# Generate the summary using the pipeline we have created
summary = summarizer(input_text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
bullet_points = summary.split(". ")
for point in bullet_points:
    print(f"- {point}")

# Print the generated summary
print("Summary:", summary)

Enter the text you want to summarize: Charles Babbage, an English mechanical engineer and polymath, originated the concept of a programmable computer. Considered the "father of the computer",[22] he conceptualized and invented the first mechanical computer in the early 19th century.  After working on his difference engine he announced his invention in 1822, in a paper to the Royal Astronomical Society, titled "Note on the application of machinery to the computation of astronomical and mathematical tables",[23] he also designed to aid in navigational calculations, in 1833 he realized that a much more general design, an analytical engine, was possible. The input of programs and data was to be provided to the machine via punched cards, a method being used at the time to direct mechanical looms such as the Jacquard loom. For output, the machine would have a printer, a curve plotter and a bell. The machine would also be able to punch numbers onto cards to be read in later. The Engine incorp

Now its time to evaluate the model . We use **ROUGE scores** for performance evaluation.  
ROUGE(*Recall-Oriented Understudy for Gisting Evaluation*) score calculates the similarity between a user provided document and machine genrated document .  

In [8]:
# first install the rouge-score package
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=0065bc5fbabf435e2238e5567d8106a84c6b7131e73cfb9b5484589b99a4bf71
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [9]:
# intall rouge_scorer from the rouge-score package
from rouge_score import rouge_scorer

In [10]:
# Define reference summaries i.e.  the same text we summarise by using the model
reference_summaries = """Charles Babbage, an English mechanical engineer and polymath, originated the concept of a programmable computer. Considered the "father of the computer",[22] he conceptualized and invented the first mechanical computer in the early 19th century.

After working on his difference engine he announced his invention in 1822, in a paper to the Royal Astronomical Society, titled "Note on the application of machinery to the computation of astronomical and mathematical tables",[23] he also designed to aid in navigational calculations, in 1833 he realized that a much more general design, an analytical engine, was possible. The input of programs and data was to be provided to the machine via punched cards, a method being used at the time to direct mechanical looms such as the Jacquard loom. For output, the machine would have a printer, a curve plotter and a bell. The machine would also be able to punch numbers onto cards to be read in later. The Engine incorporated an arithmetic logic unit, control flow in the form of conditional branching and loops, and integrated memory, making it the first design for a general-purpose computer that could be described in modern terms as Turing-complete"""

In [11]:
# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summaries, summary)

In [12]:
# Print ROUGE scores
print("Generated Summary:", summary)
print("\nROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])

Generated Summary: he invented the first mechanical computer in the early 19th century . the machine would have a printer, a curve plotter and a bell . it would be able to punch numbers onto cards to be read in later .

ROUGE-1: Score(precision=1.0, recall=0.18877551020408162, fmeasure=0.31759656652360513)
ROUGE-2: Score(precision=0.8611111111111112, recall=0.15897435897435896, fmeasure=0.2683982683982684)
ROUGE-L: Score(precision=0.972972972972973, recall=0.1836734693877551, fmeasure=0.3090128755364807)


ROUGE-1(**Unigram overlap**)  
precision = 1.0 ,it means that every u igram in the genrated summary is also
present in the refrence_summaries.

recall = 0.1887 , it means that only nearly 18% of the unigrams in the refrence_summaries are also present in the genrated summary.

fmeasure = 0.3175 ,  F-Measure of 0.3175 indicates the overall quality of the generated summary with respect to unigrams which indicates good balance between precision and recall when considering unigrams  

ROUGE-2(**Bigram overlapping**)  
These scores indicates how well the genrated summary matches the refrence_summaries in term of bigrams .

ROUGE-L(**Longest common Subsequence**)  
It measures the longest matching sequence of words between the genrated and refrence_summary

**Summary** : these score indicates that genrated_sumaary has high precision but the recall is relatively low . the fmeasures provide the balance between precision and recall . This suggest that the genrated_summary might be concise and relavant but it also missing some important informaton from the refrence_summary .

Similarly build a new text summarization using "bart-base" model as we have already done above and the we see the performace of the model .

In [13]:
summarizer2 = pipeline(
    task = "summarization",
    model = "facebook/bart-base",
    min_length = 20,
    max_length = 50,
    truncation = True,
    model_kwargs={"cache_dir":'/Documents/Hugging_Face/'}
)

In [15]:
input_text = input("Enter the text you want to summarize : ")

summary2 = summarizer2(input_text,max_length=150,min_length=50,do_sample=False)[0]['summary_text']

bullet_points2 = summary2.split(".")
for point in bullet_points2:
  print(f" -{point}")

print("Summary is : ",summary2)


Enter the text you want to summarize : Early computing machines had fixed programs. Changing its function required the re-wiring and re-structuring of the machine.[46] With the proposal of the stored-program computer this changed. A stored-program computer includes by design an instruction set and can store in memory a set of instructions (a program) that details the computation. The theoretical basis for the stored-program computer was laid out by Alan Turing in his 1936 paper. In 1945, Turing joined the National Physical Laboratory and began work on developing an electronic stored-program digital computer. His 1945 report "Proposed Electronic Calculator" was the first specification for such a device. John von Neumann at the University of Pennsylvania also circulated his First Draft of a Report on the EDVAC in 1945.[33]  The Manchester Baby was the world's first stored-program computer. It was built at the University of Manchester in England by Frederic C. Williams, Tom Kilburn and Ge

In [16]:
reference_summaries2 = """Early computing machines had fixed programs. Changing its function required the re-wiring and re-structuring of the machine.[46] With the proposal of the stored-program computer this changed. A stored-program computer includes by design an instruction set and can store in memory a set of instructions (a program) that details the computation. The theoretical basis for the stored-program computer was laid out by Alan Turing in his 1936 paper. In 1945, Turing joined the National Physical Laboratory and began work on developing an electronic stored-program digital computer. His 1945 report "Proposed Electronic Calculator" was the first specification for such a device. John von Neumann at the University of Pennsylvania also circulated his First Draft of a Report on the EDVAC in 1945.[33]

The Manchester Baby was the world's first stored-program computer. It was built at the University of Manchester in England by Frederic C. Williams, Tom Kilburn and Geoff Tootill, and ran its first program on 21 June 1948.[59] It was designed as a testbed for the Williams tube, the first random-access digital storage device.[60] Although the computer was described as "small and primitive" by a 1998 retrospective, it was the first working machine to contain all of the elements essential to a modern electronic computer.[61] As soon as the Baby had demonstrated the feasibility of its design, a project began at the university to develop it into a practically useful computer, the Manchester Mark 1."""

In [17]:
# Calculate ROUGE scores
scorer2 = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores2 = scorer.score(reference_summaries2, summary2)
# Print the generated summary and ROUGE scores
print("Generated Summary:", summary)
print("\nROUGE-1:", scores2['rouge1'])
print("ROUGE-2:", scores2['rouge2'])
print("ROUGE-L:", scores2['rougeL'])

Generated Summary: he invented the first mechanical computer in the early 19th century . the machine would have a printer, a curve plotter and a bell . it would be able to punch numbers onto cards to be read in later .

ROUGE-1: Score(precision=1.0, recall=0.5, fmeasure=0.6666666666666666)
ROUGE-2: Score(precision=0.983739837398374, recall=0.4898785425101215, fmeasure=0.6540540540540541)
ROUGE-L: Score(precision=0.9919354838709677, recall=0.4959677419354839, fmeasure=0.6612903225806452)


**Summary** : According to the scores we conclude that the genrated_summary has a high precision (*that means when it includes a word, it often relevant*) and moderate recall (*that means it captures a good portion of the content from the summary*) and fmeasures provides a balanced evaluation .  
**This indicates that the genrated_summary is quite relevant and captures asignificant portion of the content from thr reference_summary**  

Similarly build a new text summarization using "bart-large-xsum" model as we have already done above and the we see the performace of the model .

In [18]:


summarizer3 = pipeline(
    task = "summarization",
    model = "facebook/bart-large-xsum",

    min_length = 20,
    max_length = 50,
    truncation = True,
    model_kwargs={"cache_dir":'/Documents/Hugging_Face/'}
)


In [19]:
input_text = input("Enter the text you want to summarize : ")

summary3 = summarizer3(input_text,max_length=300,min_length=50,do_sample=False)[0]['summary_text']

bullet_points3 = summary3.split(".")
for point in bullet_points3:
  print(f" -{point}")

print("Summary is : ",summary3)


Enter the text you want to summarize : The concept of a field-effect transistor was proposed by Julius Edgar Lilienfeld in 1925. John Bardeen and Walter Brattain, while working under William Shockley at Bell Labs, built the first working transistor, the point-contact transistor, in 1947, which was followed by Shockley's bipolar junction transistor in 1948.[65][66] From 1955 onwards, transistors replaced vacuum tubes in computer designs, giving rise to the "second generation" of computers. Compared to vacuum tubes, transistors have many advantages: they are smaller, and require less power than vacuum tubes, so give off less heat. Junction transistors were much more reliable than vacuum tubes and had longer, indefinite, service life. Transistorized computers could contain tens of thousands of binary logic circuits in a relatively compact space. However, early junction transistors were relatively bulky devices that were difficult to manufacture on a mass-production basis, which limited th

In [20]:
refrence_summaries3 = """The concept of a field-effect transistor was proposed by Julius Edgar Lilienfeld in 1925. John Bardeen and Walter Brattain, while working under William Shockley at Bell Labs, built the first working transistor, the point-contact transistor, in 1947, which was followed by Shockley's bipolar junction transistor in 1948.[65][66] From 1955 onwards, transistors replaced vacuum tubes in computer designs, giving rise to the "second generation" of computers. Compared to vacuum tubes, transistors have many advantages: they are smaller, and require less power than vacuum tubes, so give off less heat. Junction transistors were much more reliable than vacuum tubes and had longer, indefinite, service life. Transistorized computers could contain tens of thousands of binary logic circuits in a relatively compact space. However, early junction transistors were relatively bulky devices that were difficult to manufacture on a mass-production basis, which limited them to a number of specialized applications.[67]

At the University of Manchester, a team under the leadership of Tom Kilburn designed and built a machine using the newly developed transistors instead of valves.[68] Their first transistorized computer and the first in the world, was operational by 1953, and a second version was completed there in April 1955. However, the machine did make use of valves to generate its 125 kHz clock waveforms and in the circuitry to read and write on its magnetic drum memory, so it was not the first completely transistorized computer. That distinction goes to the Harwell CADET of 1955,[69] built by the electronics division of the Atomic Energy Research Establishment at Harwell."""

In [21]:
# Calculate ROUGE scores
scorer3 = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores3 = scorer.score(refrence_summaries3, summary3)
# Print the generated summary and ROUGE scores
print("Generated Summary:", summary)
print("\nROUGE-1:", scores3['rouge1'])
print("ROUGE-2:", scores3['rouge2'])
print("ROUGE-L:", scores3['rougeL'])

Generated Summary: he invented the first mechanical computer in the early 19th century . the machine would have a printer, a curve plotter and a bell . it would be able to punch numbers onto cards to be read in later .

ROUGE-1: Score(precision=0.6382978723404256, recall=0.11406844106463879, fmeasure=0.1935483870967742)
ROUGE-2: Score(precision=0.15217391304347827, recall=0.026717557251908396, fmeasure=0.045454545454545456)
ROUGE-L: Score(precision=0.40425531914893614, recall=0.07224334600760456, fmeasure=0.12258064516129032)


**Summary** : These scores suggest that the generated summary has a relatively low recall, meaning it doesn't capture a large portion of the content from the reference summary. The precision is higher, indicating that when it includes a word, it's often relevant .
**This indicates that the generated summary is somewhat relevant and there is significant room for improvement in capturing more content from the reference summary.**

# Conclusion :

Based on the ROUGE scores, **facebook/bart-base** model has the highest ROUGE scores across all metrics (ROUGE-1, ROUGE-2, and ROUGE-L). This indicates that "facebook/bart-base" model performs the best in terms of unigram overlap, bigram overlap, and longest common subsequence compared to the other results.

The reason for this could be that it achieves a good balance between precision and recall, capturing a significant portion of the content from the reference summary while maintaining a high level of accuracy in word selection